In [1]:
# Import libraries

import sklearn as skl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

In [14]:
# Load input csv (energia and meteo)
df_energia_2021 = pd.read_csv('../input/energia_202109-202112.csv', encoding='latin-1', na_filter=False, names=["data", "hora", "normal", "horario", "autoconsumo", "injecao"], header=0)
df_energia_2022 = pd.read_csv('../input/energia_202201-202212.csv', encoding='latin-1', na_filter=False, names=["data", "hora", "normal", "horario", "autoconsumo", "injecao"], header=0)
df_meteo_2021 = pd.read_csv('../input/meteo_202109-202112.csv', encoding='latin-1', na_values=["", "NaN", ""])
df_meteo_2022 = pd.read_csv('../input/meteo_202201-202212.csv', encoding='latin-1', na_values=["", "NaN", ""])

train_energia = pd.concat([df_energia_2021, df_energia_2022])
train_meteo = pd.concat([df_meteo_2021, df_meteo_2022])

test_energia_X = pd.read_csv('../input/energia_202301-202304.csv', encoding='latin-1')
test_meteo_X = pd.read_csv('../input/meteo_202301-202304.csv', encoding='latin-1')

In [13]:
def remove_utc(s):
    if 'UTC' in s:
        return s[0:-4]
    return s

print(df_energia_2021.info())
print(df_energia_2022.info())
print(train_energia.info())

train_energia = train_energia.rename(columns={"Data":"data", "Hora":"hora", "Normal (kWh)":"normal", "Horário Económico (kWh)":"horario", "Autoconsumo (kWh)":"autoconsumo", "Injeï¿½ï¿½o na rede (kWh)":"injecao"})

print(train_energia.info())

# remove UTC from the end and transform to datetime
train_meteo['dt_iso'] = pd.to_datetime(train_meteo['dt_iso'].map(remove_utc))

# extract date and hour to different columns
train_meteo['data'] = train_meteo['dt_iso'].dt.date
train_meteo['hora'] = train_meteo['dt_iso'].dt.hour

# make date be of type date
train_energia['data'] = pd.to_datetime(train_energia['data']).dt.date

<class 'pandas.core.frame.DataFrame'>
Index: 2256 entries, 0 to 2255
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Data                     2256 non-null   object 
 1   Hora                     2256 non-null   int64  
 2   Normal (kWh)             2256 non-null   float64
 3   Horário Económico (kWh)  2256 non-null   float64
 4   Autoconsumo (kWh)        2256 non-null   float64
 5   Injeção na rede (kWh)    2256 non-null   object 
dtypes: float64(3), int64(1), object(2)
memory usage: 123.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 8760 entries, 0 to 8759
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Data                         8760 non-null   object 
 1   Hora                         8760 non-null   int64  
 2   Normal (kWh)                 8760 non-null   float64
 3   Ho

In [68]:
train = pd.merge(train_energia, train_meteo)

In [69]:
train

,data,hora,normal,horario,autoconsumo,Injeção na rede (kWh),Horï¿½rio Econï¿½mico (kWh),injecao,dt,dt_iso,...,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,wind_speed,rain_1h,clouds_all,weather_description
0,2021-09-29,0,0.000,0.0,0.0,None,NaN,NaN,1632873600,2021-09-29 00:00:00+00:00,...,11.45,14.04,1027,NaN,NaN,81,1.96,NaN,87,overcast clouds
1,2021-09-29,1,0.000,0.0,0.0,None,NaN,NaN,1632877200,2021-09-29 01:00:00+00:00,...,13.43,13.90,1028,NaN,NaN,82,1.83,NaN,91,overcast clouds
2,2021-09-29,2,0.000,0.0,0.0,None,NaN,NaN,1632880800,2021-09-29 02:00:00+00:00,...,12.72,13.43,1027,NaN,NaN,84,1.96,NaN,93,overcast clouds
3,2021-09-29,3,0.000,0.0,0.0,None,NaN,NaN,1632884400,2021-09-29 03:00:00+00:00,...,10.34,12.93,1027,NaN,NaN,85,1.85,NaN,95,overcast clouds
4,2021-09-29,4,0.000,0.0,0.0,None,NaN,NaN,1632888000,2021-09-29 04:00:00+00:00,...,9.79,12.93,1027,NaN,NaN,86,1.83,NaN,93,overcast clouds
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11011,2022-12-31,19,1.693,NaN,0.0,NaN,0.000,None,1672513200,2022-12-31 19:00:00+00:00,...,15.57,15.72,1018,NaN,NaN,86,5.56,3.58,100,moderate rain
11012,2022-12-31,20,1.327,NaN,0.0,NaN,0.000,None,1672516800,2022-12-31 20:00:00+00:00,...,12.72,15.70,1018,NaN,NaN,87,4.47,4.20,100,heavy intensity rain
11013,2022-12-31,21,0.757,NaN,0.0,NaN,0.000,None,1672520400,2022-12-31 21:00:00+00:00,...,12.23,13.99,1019,NaN,NaN,93,3.29,4.23,100,heavy intensity rain
11014,2022-12-31,22,0.000,NaN,0.0,NaN,0.675,None,1672524000,2022-12-31 22:00:00+00:00,...,12.23,13.43,1019,NaN,NaN,94,1.49,3.90,100,moderate rain


In [70]:
'''
Inspect data
'''
print(train_energia.columns)
print(train_meteo.columns)


Index(['data', 'hora', 'normal', 'horario', 'autoconsumo',
       'Injeção na rede (kWh)', 'Horï¿½rio Econï¿½mico (kWh)', 'injecao'],
      dtype='object')
Index(['dt', 'dt_iso', 'city_name', 'temp', 'feels_like', 'temp_min',
       'temp_max', 'pressure', 'sea_level', 'grnd_level', 'humidity',
       'wind_speed', 'rain_1h', 'clouds_all', 'weather_description', 'data',
       'hora'],
      dtype='object')


In [71]:

train_energia.head()

,data,hora,normal,horario,autoconsumo,Injeção na rede (kWh),Horï¿½rio Econï¿½mico (kWh),injecao
0,2021-09-29,0,0.0,0.0,0.0,None,NaN,NaN
1,2021-09-29,1,0.0,0.0,0.0,None,NaN,NaN
2,2021-09-29,2,0.0,0.0,0.0,None,NaN,NaN
3,2021-09-29,3,0.0,0.0,0.0,None,NaN,NaN
4,2021-09-29,4,0.0,0.0,0.0,None,NaN,NaN


In [72]:
train_meteo.head()

,dt,dt_iso,city_name,temp,feels_like,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,wind_speed,rain_1h,clouds_all,weather_description,data,hora
0,1630454400,2021-09-01 00:00:00+00:00,local,18.74,18.84,15.72,20.34,1015,NaN,NaN,83,1.18,NaN,78,broken clouds,2021-09-01,0
1,1630458000,2021-09-01 01:00:00+00:00,local,18.73,18.83,15.72,20.34,1014,NaN,NaN,83,1.46,NaN,92,overcast clouds,2021-09-01,1
2,1630461600,2021-09-01 02:00:00+00:00,local,17.84,17.98,16.72,20.34,1014,NaN,NaN,88,1.05,NaN,91,overcast clouds,2021-09-01,2
3,1630465200,2021-09-01 03:00:00+00:00,local,18.27,18.40,16.68,20.34,1014,NaN,NaN,86,0.46,0.14,94,light rain,2021-09-01,3
4,1630468800,2021-09-01 04:00:00+00:00,local,17.81,17.97,16.12,20.34,1013,NaN,NaN,89,0.93,1.26,95,moderate rain,2021-09-01,4
